In [1]:
from __future__ import print_function
import pickle
with open('cache/df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)

In [2]:
from collections import Counter
labels = df_test['label']
label_counter = Counter(labels)  # dict<label, #_of_occurrence>
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy

In [25]:
import numpy as np
def get_accuracy(Y, threshold=0, normalize=True):
    """
    Calculates the accuracy of a prediction, decomposed to labels. Numerical predictions are thresholded.
    Type 0: leaf accuracy, as used in Deng et al, p13
    Type 1: prediction IS the full hierarchy
    Type 2: prediction COVERS the full hierarchy
    Type 3: prediction IS any partial hierarchy
    Type 4: prediction COVERS any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    Note that type 2 and 3 are mutually exclusive.
    Args:
        Y: N * D matrix of prediction. Data type may be either numerical or boolean.
        threshold: binarize numerical prediction for accuracy type 1-4. Not needed for binary prediction.
        normalize: if False, returns positive count instead of accuracy.
    Returns:
        21 * 5 matrix, where the last row denotes overall accuracy.
    """
    is_bool = Y.dtype == np.bool
    by_label = np.zeros((20, 5), dtype=int)
    for i in range(0, len(Y)):
        l, y = labels[i], Y[i]
        if (is_bool and y[l]) or (not is_bool and np.argmax(y[:20]) == l):
            # for boolean y from CRF, leaf nodes compete in state space; for numerical y, leaf nodes compete explicitly
            by_label[l, 0] += 1
        y = np.nonzero((y / y.max()) > threshold)[0]  # predicted values are normalized to (-inf, 1]
        if tuple(y) == id_fh[l]:
            by_label[l, 1:] += 1
        elif set(id_fh[l]) in set(y):
            by_label[l, [2, 4]] += 1
        elif tuple(y) in id_hls[l]:
            by_label[l, [3, 4]] += 1
        elif any(set(x) in set(y) for x in id_hls[l]):
            by_label[l, 4] += 1
    overall = np.sum(by_label, axis=0, dtype=int)
    if normalize:
        by_label = by_label.astype(np.float32)
        for i in range(0, 20):
            by_label[i] /= label_counter[i]
        overall = overall.astype(np.float32) / len(Y)
    return np.vstack((by_label, overall))

In [12]:
np.set_printoptions(threshold=np.nan, precision=4)
id_name = hex_data['id_name']
def accuracy_str(accuracy):
    lines = str(accuracy).splitlines()
    lines = map(lambda x: x.replace(' [', '['), lines)
    lines[0] = lines[0].replace('[[', '[')
    for i in range(0, 20):
        lines[i] = '{} {}'.format(lines[i], id_name[i])
    lines[-1] = '{} {}'.format(lines[-1].replace(']]', ']'), 'overall')
    return '\n'.join(lines)

In [15]:
def opt_model_threshold(model_Y):
    """
    Performes an exaustive search for the model and threshold, as used in @get_accuracy.
    The range of search is [-0.5, 0.5], with step size 0.1.
    Args:
        model_Y: M * N * D matrix, where M is the number of models.
    Returns:
        opt_model: index of the optimal model in [0, M).
        thresholds[opt_thres]: the optimal threshold.
    """
    thresholds = np.linspace(-0.5, 0.5, num=101)
    M, T = len(model_Y), len(thresholds)
    scores = np.empty((M, T), dtype=np.float32)
    for i in range(0, M):
        for j in range(0, T):
            print('model={0}, threshold={1:.2f}'.format(i, thresholds[j]), end='\r')
            scores[i, j] = get_accuracy(model_Y[i], thresholds[j])[-1, [0, 1, 3]].prod()
    print(scores)
    opt_model, opt_thres = np.unravel_index(scores.argmax(), scores.shape)
    return opt_model, thresholds[opt_thres]

In [6]:
state_space = hex_data['state_space']
def to_crf(Y, leaf_weight=4):
    w = np.hstack(([leaf_weight] * 20, [1] * (len(id_name) - 20)))
    def to_crf_step(y):
        scores = map(lambda s: (w * y)[s].sum(), state_space)
        return state_space[np.argmax(scores)]
    return np.array(map(to_crf_step, Y), dtype=np.bool)

In [26]:
# caffe accuracy
iters = np.linspace(5000, 50000, 10, dtype=int)
iter_Y = np.load('results/test_caffe_1.npy')
# iter_Y = np.tanh(iter_Y)
# opt_iter, opt_thres = opt_model_threshold(iter_Y)
opt_iter, opt_thres = 9, 0.32
print("opt_iter={0}, opt_threshold={1:.2f}".format(iters[opt_iter], opt_thres))
print(accuracy_str(get_accuracy(iter_Y[opt_iter], threshold=opt_thres)))

opt_iter=50000, opt_threshold=0.32
[ 0.0256  0.      0.      0.0256  0.0256] diningtable
[ 0.0185  0.      0.      0.0185  0.0185] chair
[ 0.012   0.      0.      0.0482  0.0482] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.0851  0.      0.      0.      0.    ] tvmonitor
[ 0.0561  0.      0.      0.1589  0.1589] train
[ 0.0132  0.0132  0.0132  0.1316  0.1316] bus
[ 0.0719  0.036   0.036   0.1367  0.1367] car
[ 0.0208  0.0208  0.0208  0.125   0.125 ] bicycle
[ 0.037   0.0185  0.0185  0.1852  0.1852] motorbike
[ 0.1186  0.0593  0.0593  0.0678  0.0678] aeroplane
[ 0.0278  0.      0.      0.0139  0.0139] boat
[ 0.      0.      0.      0.      0.    ] cow
[ 0.      0.      0.      0.0154  0.0154] horse
[ 0.      0.      0.      0.0303  0.0303] sheep
[ 0.0964  0.006   0.006   0.2289  0.2289] dog
[ 0.1282  0.0321  0.0321  0.2628  0.2628] cat
[ 0.0227  0.      0.      0.2121  0.2121] bird
[ 0.5     0.0506  0.0506  0.0618  0.061

In [36]:
# caffe + crf accuracy
iters = np.linspace(5000, 50000, 10, dtype=np.uint16)
iter_Y = np.load('results/test_caffe_4.npy')
iter_Y = np.tanh(iter_Y)
opt_iter = opt_model(map(to_crf, iter_Y))
# np.argmax([get_accuracy(Y)[-1, 0] for Y in model_Y])
print('opt_iter={}'.format(iters[opt_iter]))
print(accuracy_str(get_accuracy(to_crf(iter_Y[opt_iter]))))

opt_iter=50000
[ 0.0513  0.0513  0.0513  0.0513  0.0513] diningtable
[ 0.0185  0.0185  0.0185  0.0185  0.0185] chair
[ 0.      0.      0.      0.      0.    ] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.      0.      0.      0.      0.    ] tvmonitor
[ 0.0654  0.0654  0.0654  0.0654  0.0654] train
[ 0.0263  0.0263  0.0263  0.0263  0.0263] bus
[ 0.1295  0.1295  0.1295  0.1295  0.1295] car
[ 0.0417  0.0417  0.0417  0.0417  0.0417] bicycle
[ 0.037   0.037   0.037   0.037   0.037 ] motorbike
[ 0.0678  0.0678  0.0678  0.0678  0.0678] aeroplane
[ 0.0139  0.0139  0.0139  0.0139  0.0139] boat
[ 0.      0.      0.      0.      0.    ] cow
[ 0.0462  0.0462  0.0462  0.0462  0.0462] horse
[ 0.      0.      0.      0.      0.    ] sheep
[ 0.0663  0.0663  0.0663  0.0663  0.0663] dog
[ 0.1667  0.1667  0.1667  0.1667  0.1667] cat
[ 0.0227  0.0227  0.0227  0.0227  0.0227] bird
[ 0.6067  0.6067  0.6067  0.6067  0.6067] person
[ 0.1152  

In [21]:
# sam setting, different result???
# svm accuracy
kernels = ['linear', 'poly', 'rbf']
kernel_Y = np.load('results/test_svm_4.npy')
# kernel_Y = np.log(kernel_Y / (1 - kernel_Y))  # log ratio
# opt_kernel, opt_thres = opt_model_threshold(kernel_Y)
opt_kernel, opt_thres = 2, 0.27
print("opt_kernel={0}, opt_threshold={1:.2f}".format(kernels[opt_kernel], opt_thres))
print(accuracy_str(get_accuracy(kernel_Y[opt_kernel], threshold=opt_thres)))

opt_kernel=rbf, opt_threshold=0.27
[ 0.0256  0.      0.      0.0256  0.0256] diningtable
[ 0.      0.      0.      0.0185  0.0185] chair
[ 0.0241  0.012   0.012   0.0602  0.0602] sofa
[ 0.      0.      0.      0.      0.    ] bottle
[ 0.      0.      0.      0.      0.    ] pottedplant
[ 0.      0.      0.      0.      0.    ] tvmonitor
[ 0.0467  0.0187  0.0187  0.1028  0.1028] train
[ 0.0132  0.0132  0.0132  0.0526  0.0526] bus
[ 0.0504  0.0432  0.0432  0.1583  0.1583] car
[ 0.0417  0.0208  0.0208  0.2083  0.2083] bicycle
[ 0.0556  0.0556  0.0556  0.2037  0.2037] motorbike
[ 0.0847  0.0508  0.0508  0.0678  0.0678] aeroplane
[ 0.0556  0.0139  0.0139  0.0694  0.0694] boat
[ 0.      0.      0.      0.0625  0.0625] cow
[ 0.0154  0.0154  0.0154  0.0769  0.0769] horse
[ 0.      0.      0.      0.0303  0.0303] sheep
[ 0.1024  0.0422  0.0422  0.1627  0.1627] dog
[ 0.1795  0.109   0.109   0.2372  0.2372] cat
[ 0.0833  0.0227  0.0227  0.2197  0.2197] bird
[ 0.8202  0.5225  0.5225  0.5955  0.595

In [40]:
# svm + crf accuracy
kernels = ['linear', 'poly', 'rbf']
kernel_Y = np.load('results/test_svm_1.npy')
# kernel_Y = np.log(kernel_Y / (1 - kernel_Y))  # log ratio
opt_kernel = opt_model(map(to_crf, kernel_Y))
# np.argmax([get_accuracy(Y)[-1, 0] for Y in model_Y])
print('opt_kernel={}'.format(kernels[opt_kernel]))
print(accuracy_str(get_accuracy(to_crf(kernel_Y[opt_kernel]))))

opt_kernel=poly
[ 0.      0.      0.      0.1282  0.1282] diningtable
[ 0.      0.      0.      0.037   0.037 ] chair
[ 0.      0.      0.      0.1084  0.1084] sofa
[ 0.      0.      0.      0.0263  0.0263] bottle
[ 0.      0.      0.      0.0323  0.0323] pottedplant
[ 0.      0.      0.      0.0638  0.0638] tvmonitor
[ 0.0187  0.0187  0.0187  0.1869  0.1869] train
[ 0.0132  0.0132  0.0132  0.1447  0.1447] bus
[ 0.0216  0.0216  0.0216  0.2158  0.2158] car
[ 0.0417  0.0417  0.0417  0.2292  0.2292] bicycle
[ 0.0556  0.0556  0.0556  0.2037  0.2037] motorbike
[ 0.0424  0.0424  0.0424  0.178   0.178 ] aeroplane
[ 0.      0.      0.      0.0833  0.0833] boat
[ 0.      0.      0.      0.2292  0.2292] cow
[ 0.      0.      0.      0.1846  0.1846] horse
[ 0.      0.      0.      0.1818  0.1818] sheep
[ 0.0422  0.0422  0.0422  0.2892  0.2892] dog
[ 0.1154  0.1154  0.1154  0.3526  0.3526] cat
[ 0.0227  0.0227  0.0227  0.2576  0.2576] bird
[ 0.5     0.5     0.5     0.7697  0.7697] person
[ 0.079  